In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import uuid

from datasets import Dataset

In [26]:
url = "https://xn----dtbrojdkckkfj9k.xn--p1ai/vopros-yuristu?page=560"
response = requests.get(url=url)
soup = BeautifulSoup(response.text)
items = soup.find("div", id="block-system-main")
items = items.find("div", class_="view-content")
items = items.find_all("li", class_="views-row")
len(items), items[0]

(4,
 <li class="views-row views-row-1 views-row-odd views-row-first"> <article about="/content/%D0%B8%D0%B7%D1%8A%D1%8F%D1%82%D0%B8%D0%B5-%D0%B8%D0%BF%D0%BE%D1%82%D0%B5%D1%87%D0%BD%D0%BE%D0%B9-%D0%BA%D0%B2%D0%B0%D1%80%D1%82%D0%B8%D1%80%D1%8B" class="node node-forum node-teaser clearfix" id="node-153" typeof="sioc:Post sioct:BoardPost">
 <header>
 <h2 class="title"><a href="/content/%D0%B8%D0%B7%D1%8A%D1%8F%D1%82%D0%B8%D0%B5-%D0%B8%D0%BF%D0%BE%D1%82%D0%B5%D1%87%D0%BD%D0%BE%D0%B9-%D0%BA%D0%B2%D0%B0%D1%80%D1%82%D0%B8%D1%80%D1%8B">изъятие ипотечной квартиры</a></h2>
 <span class="rdf-meta element-hidden" content="изъятие ипотечной квартиры" property="dc:title"></span><span class="rdf-meta element-hidden" content="1" datatype="xsd:integer" property="sioc:num_replies"></span>
 <div class="submitted"><span content="2018-03-29T16:04:35+03:00" datatype="xsd:dateTime" property="dc:date dc:created" rel="sioc:has_creator">Опубликовано чт, 03/29/2018 - 16:04 пользователем <a about="/users/alexlawye

In [25]:
link = items[0].find("h2").find("a")["href"]
link = f"https://xn----dtbrojdkckkfj9k.xn--p1ai{link}"
link

'https://xn----dtbrojdkckkfj9k.xn--p1ai/content/%D0%B8%D0%B7%D1%8A%D1%8F%D1%82%D0%B8%D0%B5-%D0%B8%D0%BF%D0%BE%D1%82%D0%B5%D1%87%D0%BD%D0%BE%D0%B9-%D0%BA%D0%B2%D0%B0%D1%80%D1%82%D0%B8%D1%80%D1%8B'

In [28]:
from tqdm import tqdm


class LawForumLinksParser:
    def __init__(
        self,
    ) -> None:
        self.dataset = []
        self.domain = "https://xn----dtbrojdkckkfj9k.xn--p1ai"

    def parse(
        self,
    ):
        pages_amount = 571
        for page_num in tqdm(range(pages_amount)):
            url = f"{self.domain}/vopros-yuristu?page={page_num}"
            response = requests.get(url=url)
            soup = BeautifulSoup(response.text)
            items = soup.find("div", id="block-system-main")
            items = items.find("div", class_="view-content")
            items = items.find_all("li", class_="views-row")

            for item in items:
                link = item.find("h2").find("a")["href"]
                link = f"{self.domain}{link}"
                self.dataset.append({"link": link})

        pd.DataFrame(data=self.dataset).to_csv(
            "./verbalist/datasets/forum_uristov_rf/forum_uristov_rf_links.csv",
            index=False,
        )


links_parser = LawForumLinksParser()
links_parser.parse()

100%|██████████| 571/571 [00:44<00:00, 12.85it/s]


In [ ]:
import markdownify

url = "https://xn----dtbrojdkckkfj9k.xn--p1ai/content/%D0%B8%D0%B7%D1%8A%D1%8F%D1%82%D0%B8%D0%B5-%D0%B8%D0%BF%D0%BE%D1%82%D0%B5%D1%87%D0%BD%D0%BE%D0%B9-%D0%BA%D0%B2%D0%B0%D1%80%D1%82%D0%B8%D1%80%D1%8B"
response = requests.get(url=url)
soup = BeautifulSoup(response.text)
items = soup.find_all("div", class_="forum-post-content")

In [42]:
question = (
    str(markdownify.markdownify(str(items[0]))).replace("Ваш вопрос:", "").strip()
)

print(question)

Здравствуйте, имеет ли права судебные приставы или банк забрать ипотечную квартиры если ее часть оплачена материнским капиталом.


In [40]:
response = str(markdownify.markdownify(str(items[1]))).replace("\n\n", "\n").strip()
print(response)

Взыскание на ипотечную квартиру невозможно в следующих случаях:

Согласно статье 348 Гражданского кодекса РФ:

- взыскание на заложенное имущество для удовлетворения требований залогодержателя может быть обращено в случае неисполнения или ненадлежащего исполнения должником обеспеченного залогом обязательства (п. 1);

- обращение взыскания на заложенное имущество не допускается, если допущенное должником нарушение обеспеченного залогом обязательства незначительно и размер требований залогодержателя вследствие этого явно несоразмерен стоимости заложенного имущества. Если не доказано иное, предполагается, что нарушение обеспеченного залогом обязательства незначительно и размер требований залогодержателя явно несоразмерен стоимости заложенного имущества при условии, что одновременно соблюдены следующие условия:

Если ответчики распорядились материнским капиталом для улучшения жилищных условий семьи, малолетних детей, а обращение взыскания на заложенную квартиру с её реализацией может послу

In [47]:
class ContentParser:
    def __init__(self, urls: list[str]) -> None:
        self.urls = urls

        self.dataset = []

    def parse(self):
        for url in tqdm(self.urls):
            # print(url)
            response = requests.get(url=url)
            soup = BeautifulSoup(response.text)
            items = soup.find_all("div", class_="forum-post-content")
            if len(items) == 2:
                question = (
                    str(markdownify.markdownify(str(items[0])))
                    .replace("Ваш вопрос:", "")
                    .strip()
                )
                answer = (
                    str(markdownify.markdownify(str(items[1])))
                    .replace("\n\n", "\n")
                    .strip()
                )
                self.dataset.append({"prompt": question, "solution": answer, "link": url})

        pd.DataFrame(data=self.dataset).to_csv(
            "./verbalist/datasets/forum_uristov_rf/forum_uristov_rf_prompts.csv",
            index=False,
        )

urls = pd.read_csv("./verbalist/datasets/forum_uristov_rf/forum_uristov_rf_links.csv")        
urls = urls['link'] 
parser = ContentParser(urls=urls)
parser.parse()

100%|██████████| 2283/2283 [09:44<00:00,  3.91it/s]


In [48]:
parser.dataset[0]

{'prompt': 'Регион:\xa0[Россия](/regiony/rossiya)\xa0Доброе утро.\r\nПокупаю приватизированную квартиру. На момент приватизации зарегистрированы в квартире были четверо — мать, отец, двое совершеннолетних сыновей. Сыновья в приватизации не участвовали. Один из сыновей уже снялся с учета и предоставил нотариально заверенное заявление такого содержания:\r\n«Я, Иванов Иван Иванович, настоящим заявлением подтверждаю, что мне известно, что на момент приватизации квартиры, находящейся по адресу …, и я был зарегистрирован в указанной квартире, но в число собственников включен не был. Подтверждаю, что претензий по приватизации вышеуказанной квартиры не имею, в суд по поводу признания договора передачи частично недействительным обращаться не буду. Я, Иванов Иван Иванович, настоящим заявлением не возражаю против продажи данной квартиры.»\r\nЗдесь ни слова о том, что в связи с добровольной сменой жительства его бессрочное право пользования квартирой ПРЕКРАЩЕНО. И к тому же, человек выписался "в н

In [50]:
dataset = Dataset.from_list(parser.dataset)
dataset

Dataset({
    features: ['prompt', 'solution', 'link'],
    num_rows: 1849
})

In [51]:
# dataset.push_to_hub("dim/forum_uristov_rf_prompts", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]